# Oscilloscope

In [ ]:
import oscilloscope
app = oscilloscope.oscilloscope()

In [ ]:
app.display()

In [ ]:
from IPython.lib import backgroundjobs as bg
jobs = bg.BackgroundJobManager()

jobs.new('app.run()')